In [1]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 14318, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 14318 (delta 33), reused 32 (delta 10), pack-reused 14242
Receiving objects: 100% (14318/14318), 13.65 MiB | 17.52 MiB/s, done.
Resolving deltas: 100% (9840/9840), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [2]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 1.6 MB 45.4 MB/s 
Setup complete. Using torch 1.12.1+cu113 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="TBSZIcJvZGLN6tqEEfeO")
project = rf.workspace("night-vision-mmcmp").project("mul")
dataset = project.version(1).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 958 kB/s 
     |████████████████████████████████| 178 kB 50.7 MB/s 
     |████████████████████████████████| 138 kB 69.9 MB/s 
     |████████████████████████████████| 54 kB 4.0 MB/s 
     |████████████████████████████████| 145 kB 67.5 MB/s 
     |████████████████████████████████| 67 kB 7.6 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=6282032461aeb2ba5c4ac74eaab1544094ea57455b4d69ae95f07215eb47bc02
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing installation: certi

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Mul-1 in yolov5pytorch:: 100%|██████████| 9593/9593 [00:02<00:00, 3213.41it/s]


In [4]:
%cd /content/yolov5
%cat {dataset.location}/data.yaml

/content/yolov5
names:
- Bird
- Drone
- Helicopter
- Plane
nc: 4
roboflow:
  license: CC BY 4.0
  project: mul
  url: https://universe.roboflow.com/night-vision-mmcmp/mul/dataset/1
  version: 1
  workspace: night-vision-mmcmp
test: ../test/images
train: Mul-1/train/images
val: Mul-1/valid/images


In [5]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [6]:
#this is the model configuration we will use for our tutorial 
%cat //content/yolov5/models/hub/yolov5-p2.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors: 3  # AutoAnchor evolves 3 anchors per P output layer

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head with (P2, P3, P4, P5) outputs
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 

In [7]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [8]:
%%writetemplate /content/yolov5/models/GAANetxl.yaml
# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.50  # model depth multiple
width_multiple: 0.75  # layer channel multiple
anchors: 3  # AutoAnchor evolves 3 anchors per P output layer

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [256, 6, 2, 2]],  # 0-P1/2
   [-1, 1, GhostConv, [512, 3, 2]],  # 1-P2/4
   [-1, 1, C3Ghost, [512]],
   [-1, 1, GhostConv, [768, 3, 2]],  # 3-P3/8
   [-1, 2, C3Ghost, [768]],
   [-1, 1, GhostConv, [1024, 3, 2]],  # 5-P4/16
   [-1, 4, C3Ghost, [1024]],
   [-1, 1, GhostConv, [1280, 3, 2]],  # 7-P5/32
   [-1, 6, C3Ghost, [1280]],
   [-1, 1, SPPF, [1280, 3]],  # 9
  ]
# YOLOv5 v6.0 head with (P2, P3, P4, P5) outputs
head:
  [[-1, 1, GhostConv, [1024, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 1, C3Ghost, [1024, False]],  # 13

   [-1, 1, GhostConv, [768, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 1, C3Ghost, [768, False]],  # 17 (P3/8-small)

   [-1, 1, GhostConv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 2], 1, Concat, [1]],  # cat backbone P2
   [-1, 1, C3Ghost, [512, False]],  # 21 (P2/4-xsmall)

   [-1, 1, GhostConv, [512, 3, 2]],
   [[-1, 18], 1, Concat, [1]],  # cat head P3
   [-1, 1, C3Ghost, [768, False]],  # 24 (P3/8-small)

   [-1, 1, GhostConv, [768, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 1, C3Ghost, [1024, False]],  # 27 (P4/16-medium)

   [-1, 1, GhostConv, [1024, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 1, C3Ghost, [1280, False]],  # 30 (P5/32-large)

   [[21, 24, 27, 30], 1, Detect, [nc, anchors]],  # Detect(P2, P3, P4, P5)
  ]

In [ ]:
# train yolov5s on custom data 
%%time
%cd /content/yolov5/
!python train.py --img 256 --batch 64 --epochs 500 --optimizer AdamW --hyp data/hyps/hyp.scratch-med.yaml --data {dataset.location}/data.yaml --cfg ./models/GAANetxl.yaml --weights '' --name GAANetxl_results  --cache

/content/yolov5
train: weights=, cfg=./models/GAANetxl.yaml, data=/content/yolov5/Mul-1/data.yaml, hyp=data/hyps/hyp.scratch-med.yaml, epochs=500, batch_size=64, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=AdamW, sync_bn=False, workers=8, project=runs/train, name=GAANetxl_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 388 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, 

# Evaluate Custom YOLOv5 Detector Performance

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/GAANetxl_results/results.png', width=1000)  # view results.png

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/
%ls runs/train/GAANetxl_results/weights

In [ ]:
#########Validate whole dataset
# Validate model and print evaluation metrics
!python val.py --weights runs/train/GAANetxl_results/weights/best.pt --data /content/yolov5/Mul-1/train/images --img 256 --half

In [ ]:
######## Test Valid image folder
%cd /content/yolov5/
!python detect.py --weights runs/train/GAANetxl_results/weights/best.pt --img 256 --conf 0.25 --source /content/yolov5/Mul-1/valid/images

In [ ]:
######## Test Train image folder
%cd /content/yolov5/
# Perform detection and save results
!python detect.py --weights runs/train/GAANetxl_results/weights/best.pt --img 256 --conf 0.25 --source /content/yolov5/Mul-1/train/images

In [ ]:
##I have created a zip file:
!zip -r /content/GAANetm_results.zip /content/yolov5/runs/
##Than I have downloded that zip file:
from google.colab import files
files.download("/content/GAANetm_results.zip")